# Introduction

In this notebook we are going to implement a Question Answering system leveraging **BERT-SQuAD**.

**BERT-SQuAD** is a BERT model fine-tuned for the SQuAD (Stanford Question Answering Dataset) dataset. SQuAD is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

In layman terms, as we explained in the language modeling session, BERT is a language model that is able to understand English and BERT-SQuAD is the same model, but now fine tuned to know how to answer questions (i.e., learn the nuances related to answering questions, such as focus on the entities of the question to provide the answer). For a more detailed description of the model, please refer to the original Stanford paper: https://web.stanford.edu/class/cs224n/reports/default/15848021.pdf

In particular, we are going to apply this model to a scenario that is, sadly, very topical nowadays. To answer questions about the COVID-19. In response to the COVID-19 pandemic, the White House and a coalition of leading research groups have prepared the COVID-19 Open Research Dataset (CORD-19).
CORD-19 is a resource of over 200,000 scholarly articles, including over 100,000 with full text, about COVID-19, SARS-CoV-2, and related coronaviruses. For more information, please refer to: https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge

Together with the CORD-19 dataset, a groups of tasks (or questions) have been released: https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge/tasks. What we are trying to do is to, leveraging the BERT-SQuAD model, "read" the 200K articles and answer the questions.



Install the required environment and path.

In [ ]:
!pip install tqdm
!pip install transformers
!pip install ktrain

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ! mkdir '/content/drive/My Drive/covid-kaggle/input'

# Data Download

Let's download the dataset from Kaggle, please be aware that the dataset is large and this step could take a while.

Once downloaded the zip file, we are going to unzip it to create the folder structure with the different sources and the related articles


In [ ]:
# ! unzip -qq '/content/drive/My Drive/covid-kaggle/input/CORD-19-research-challenge.zip' -d '/content/drive/My Drive/covid-kaggle/input/'

In [ ]:
# Paths to json files
path_1 = '/content/drive/My Drive/covid-kaggle/input/biorxiv_medrxiv/biorxiv_medrxiv/pdf_json/'
path_2 = '/content/drive/My Drive/covid-kaggle/input/comm_use_subset/comm_use_subset/pdf_json/'
path_3 = '/content/drive/My Drive/covid-kaggle/input/noncomm_use_subset/noncomm_use_subset/pdf_json/'
path_4 = '/content/drive/My Drive/covid-kaggle/input/custom_license/custom_license/pdf_json/'

# List of folder names
folder_names = ['biorxiv_medrxiv','comm_use_subset']
data_path = [path_1, path_2, path_3, path_4]
# data_path = [path_1]

Import the required library.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import nltk
nltk.download('punkt')
from math import log, sqrt
from collections import defaultdict
from copy import deepcopy
from tqdm import tqdm
from IPython.core.display import display, HTML

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


If you navigate the folder structure that has been created, you will see that the articles are provided as JSON files.

The following functions allow the extraction of data from the JSONs to the dataframes

In [ ]:
# Extraction of data from json format to dataframe 
def format_name(author):
    middle_name = " ".join(author['middle'])
    if author['middle']:
        return " ".join([author['first'], middle_name, author['last']])
    else:
        return " ".join([author['first'], author['last']])

def format_affiliation(affiliation):
    text = []
    location = affiliation.get('location')
    if location:
        text.extend(list(affiliation['location'].values()))
    institution = affiliation.get('institution')
    if institution:
        text = [institution] + text
    return ", ".join(text)

def format_authors(authors, with_affiliation=False):
    name_ls = []
    for author in authors:
        name = format_name(author)
        if with_affiliation:
            affiliation = format_affiliation(author['affiliation'])
            if affiliation:
                name_ls.append(f"{name} ({affiliation})")
            else:
                name_ls.append(name)
        else:
            name_ls.append(name)
    return ", ".join(name_ls)

def format_body(body_text):
    texts = [(di['section'], di['text']) for di in body_text]
    texts_di = {di['section']: "" for di in body_text}
    for section, text in texts:
        texts_di[section] += text
    body = ""
    for section, text in texts_di.items():
        body += section
        body += "\n\n"
        body += text
        body += "\n\n"
    return body

def format_bib(bibs):
    if type(bibs) == dict:
        bibs = list(bibs.values())
    bibs = deepcopy(bibs)
    formatted = []
    for bib in bibs:
        bib['authors'] = format_authors(
            bib['authors'], 
            with_affiliation=False
        )
        formatted_ls = [str(bib[k]) for k in ['title', 'authors', 'venue', 'year']]
        formatted.append(", ".join(formatted_ls))
    return "; ".join(formatted)

def load_files(dirname, filename=None):
    filenames = os.listdir(dirname)
    raw_files = []
    if filename:
        filename = dirname + filename
        raw_files = [json.load(open(filename, 'rb'))]
    else:
        for filename in tqdm(filenames):
            filename = dirname + filename
            file = json.load(open(filename, 'rb'))
            raw_files.append(file)
    return raw_files

def generate_clean_df(all_files):
    cleaned_files = []
    
    for file in tqdm(all_files):
        features = [
            file['paper_id'],
            file['metadata']['title'],
            format_authors(file['metadata']['authors']),
            format_authors(file['metadata']['authors'], 
                           with_affiliation=True),
            format_body(file['abstract']),
            format_body(file['body_text']),
            format_bib(file['bib_entries']),
            file['metadata']['authors'],
            file['bib_entries']
        ]

        cleaned_files.append(features)

    col_names = ['paper_id', 'title', 'authors',
                 'affiliations', 'abstract', 'text', 
                 'bibliography','raw_authors','raw_bibliography']

    clean_df = pd.DataFrame(cleaned_files, columns=col_names)
    clean_df = clean_df.drop(columns=['authors','affiliations','bibliography',
                                      'raw_authors','raw_bibliography'])
    return clean_df

# Data conversion from dataframe to 2 lists
def df2list(dir_path = data_path):
  corpus = []
  paper_id = []
  for dp in dir_path:
    filenames = os.listdir(dp)
    print('Number of articles retrieved:', len(filenames))
    files = load_files(dp)
    df = generate_clean_df(files)
    corpus.extend(list(df['title'] + ' ' + df['abstract'] + ' ' + df['text']))
    paper_id.extend(list(df['paper_id']))
  return corpus, paper_id

corpus, paper_id = df2list()




  3%|▎         | 42/1625 [00:00<00:03, 418.38it/s]

Number of articles retrieved: 1625


  0%|          | 22/9524 [00:00<00:49, 190.99it/s]

Number of articles retrieved: 9524


  1%|▏         | 33/2490 [00:00<00:08, 292.13it/s]

Number of articles retrieved: 2490


  0%|          | 0/26505 [00:00<?, ?it/s]

Number of articles retrieved: 26505


 60%|█████▉    | 15846/26505 [00:53<00:31, 338.62it/s]

In [ ]:
len(corpus)

# BERT Squad Modelling

Nice, we already have the data loaded into the proper format. 
Now it's time for the funny part, the use of BERT for QA. There are different ways of using this models, however, I do recommend you to make use of the [ktrain](https://github.com/amaiya/ktrain) Python library that we know from the Text Classification Practice.

The model involves different steps. The first one is to index the research articles that we will treat as a knowledge base to which we can ask questions and retrieve exact answers using BERT. Then we will instantiate a new BERT model and finally we will define the queries to be answered by the system




In [ ]:
import ktrain
from ktrain import text

## STEP 1: Index the Documents

We will first index the documents into a search engine that will be used to quickly retrieve documents that are likely to contain answers to a question. To do so, we must choose an index location, which must be a folder that does not already exist.


In [ ]:
INDEXDIR = '/content/drive/My Drive/covid-kaggle/index-bert'
text.SimpleQA.initialize_index(INDEXDIR)
text.SimpleQA.index_from_list(corpus, INDEXDIR, commit_every=len(corpus))

The above steps need to only be performed once. Once an index is already created, you can skip this step and proceed directly to STEP 2 to begin using your system.

## STEP 2: Create a QA instance

Next, we create a QA instance. This step will automatically download the BERT SQUAD model if it does not already exist on your system.

In [ ]:
qa = text.SimpleQA(INDEXDIR)



## STEP 3: Ask Questions

We will invoke the ask method to issue questions to the text corpus we indexed and retrieve answers. We will also use the qa.display method to nicely display the top 5 results in this Jupyter notebook. The answers are inferred using a BERT model trained on the SQUAD dataset. Since the model is combing through paragraphs and sentences to find an answer, it may take a minute or two to return results.


In [ ]:
all_topics=[
    'What is known about transmission, incubation, and environmental stability?',
    'What do we know about COVID-19 risk factors?',
    'What do we know about virus genetics, origin, and evolution?',
    'What do we know about vaccines and therapeutics?',
    'What do we know about non-pharmaceutical interventions?',
    'What has been published about medical care?',
    'What do we know about diagnostics and surveillance?'
    'What has been published about information sharing and inter-sectoral collaboration?',
    'What has been published about ethical and social science considerations?'
]
topic_area = {}

#0
#What is known about transmission, incubation, and environmental stability?
question_list = []
kw_list = []
pm_kw_list = []
question_list.append("Is the virus transmitted by aerisol, droplets, food, close contact, fecal matter, or water")
kw_list.append("2019-nCoV,SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, person to person, human to human, interpersonal contact, air, water,fecal, surfaces, aerisol, transmission, shedding")
pm_kw_list.append("2019-nCoV, transmission, shedding")

question_list.append( "How long is the incubation period for the virus")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, hours, days, period")
pm_kw_list.append("2019-nCoV, incubation period")

question_list.append("Can the virus be transmitted asymptomatically or during the incubation period")
kw_list.append("2019-nCoV, COVID-19, coronavirus, novel coronavirus, asymptomatic, person to person, human to human, transmission")
pm_kw_list.append("2019-nCoV, asymptomatic, transmission")

question_list.append("What is the quantity of asymptomatic shedding")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, asymptomatic, shedding, percentage, rate, quantity, pediatric")
pm_kw_list.append("2019-nCoV, asymptomatic, shedding")

question_list.append("How does temperature and humidity affect the tramsmission of 2019-nCoV")
kw_list.append("2019-nCoV, COVID-19, coronavirus, novel coronavirus, temperature, humidity")
pm_kw_list.append("2019-nCoV, temperature, humidity")

question_list.append("How long can 2019-nCoV remain viable on inanimate, environmental, or common surfaces")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, inanimate, environmental, touch, copper, plastic, steel, wood, fabric, glass, porous, nonporous")
pm_kw_list.append("2019-nCoV, surface")

question_list.append("What types of inanimate or environmental surfaces affect transmission, survival, or  inactivation of 2019-nCov")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, inanimate, environmental, touch, copper, plastic, steel, wood, fabric, glass, porous, nonporous")
pm_kw_list.append("2019-nCoV, surface")

question_list.append("Can the virus be found in nasal discharge, sputum, urine, fecal matter, or blood")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, shedding, body fluid")
pm_kw_list.append("2019-nCoV, body fluids")

topic_area['What is known about transmission, incubation, and environmental stability?'] = list(zip(question_list,kw_list, pm_kw_list))



#1
#What do we know about COVID-19 risk factors?
question_list = []
kw_list = []
pm_kw_list = []
question_list.append("What risk factors contribute to the severity of 2019-nCoV")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, susceptible, smoking, smoker, neonates, pregnant, socio-economic, behavioral, age, elderly, young, old, children")
pm_kw_list.append("2019-nCoV, risk factors")

question_list.append("How does hypertension affect patients")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, hypertension, blood pressure, comorbidity")
pm_kw_list.append("2019-nCoV, hypertension, comorbidity")

question_list.append("How does heart disease affect patients")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, heart disease, comorbidity")
pm_kw_list.append("2019-nCoV, heart disease, comorbidity")

question_list.append("How does copd affect patients")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, copd, chronic obstructive pulmonary disease")
pm_kw_list.append("2019-nCoV, copd, chronic obstructive pulmonary disease")

question_list.append("How does smoking affect 2019-nCoV patients")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, smoking, smoker")
pm_kw_list.append("2019-nCoV, smoking, smoker")

question_list.append("How does pregnancy affect patients")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, pregnant, pregnancy")
pm_kw_list.append("2019-nCoV, pregnant, pregnancy")

question_list.append("What are the case fatality rates for 2019-nCoV patients")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, fatality rate")
pm_kw_list.append("2019-nCoV, fatality, statistics, death")

question_list.append("What is the case fatality rate in Italy")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, fatality rate, Italy")
pm_kw_list.append("2019-nCoV, fatality, statistics, death")

question_list.append("What public health policies prevent or control the spread of 2019-nCoV")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, guidance, prevention measures, public health, community, prevention, administration, government, health department, policy, control measures, travel")
pm_kw_list.append("2019-nCoV, guidance, public health,  policy, control measures")

topic_area['What do we know about COVID-19 risk factors?'] = list(zip(question_list,kw_list, pm_kw_list))


#2
#What do we know about virus genetics, origin, and evolution?
question_list = []
kw_list = []
pm_kw_list = []
question_list.append("Can animals transmit 2019-nCoV")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, animals, zoonotic, farm, spillover, animal to human, human to animal, reservoir")
pm_kw_list.append("2019-nCoV, spillover, reservoir")

question_list.append("What animal did 2019-nCoV come from")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, animals, zoonotic, farm, spillover, animal to human, bats, snakes, exotic animals")
pm_kw_list.append("2019-nCoV, zoonotic")

question_list.append("What real-time genomic tracking tools exist")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, real-time, gene, typing, tracking, software, reporting")
pm_kw_list.append('"2019-nCoV, real-time, genomic, tracking')

question_list.append("What regional genetic variations (mutations) exist")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, geography, region, genome, mutations")
pm_kw_list.append("2019-nCoV, geneome, region")

question_list.append("What effors are being done in asia to prevent further outbreaks")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, surveylance, wildlife, livestock, monitoring, asia, prevent, prevention, outbreaks")
pm_kw_list.append("2019-nCoV, surveylance")

topic_area['What do we know about virus genetics, origin, and evolution?'] = list(zip(question_list,kw_list, pm_kw_list))

#3
#What do we know about vaccines and therapeutics?
question_list = []
kw_list = []
pm_kw_list = []
question_list.append("What drugs or therapies are being investigated")
kw_list.append("2019-nCoV,  COVID-19, coronavirus, novel coronavirus, drug, antiviral, testing, clinical trial, study")
pm_kw_list.append("2019-nCoV,  drug, therapy")

question_list.append("What clinical trials for hydroxychloroquine have been completed")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, hydroxychloroquine, clinical trial")
pm_kw_list.append("2019-nCoV, hydroxychloroquine")

question_list.append("What antiviral drug clinical trials have been completed")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, clinical trial")
pm_kw_list.append("2019-nCoV, antiviral")

question_list.append("Are anti-inflammatory drugs recommended")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, ibuprofen, advil, NSAID, anti-inflamatory, treatment")
pm_kw_list.append('2019-nCoV, ibuprofen, NSAID')

topic_area['What do we know about vaccines and therapeutics?'] = list(zip(question_list,kw_list, pm_kw_list))


#4
#What do we know about non-pharmaceutical interventions?
question_list = []
kw_list = []
pm_kw_list = []
question_list.append("Which non-pharmaceutical interventions limit tramsission")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, non-pharmaceutical interventions, npi")
pm_kw_list.append("2019-nCoV, npi")

question_list.append("What are most important barriers to compliance")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, non-pharmaceutical interventions, npi")
pm_kw_list.append('2019-nCoV, npi, barrier to compliance')

topic_area['What do we know about non-pharmaceutical interventions?'] = list(zip(question_list,kw_list, pm_kw_list))

#5
#What has been published about medical care?
question_list = []
kw_list = []
pm_kw_list = []
question_list.append("How does extracorporeal membrane oxygenation affect 2019-nCoV patients")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, extracorporeal membrane oxygenation, ecmo")
pm_kw_list.append('2019-nCoV, ecmo')

question_list.append("What telemedicine and cybercare methods are most effective")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, telemedicine, 5G, cell phone, cyber, cybercare, information technolog, remote, over the phone, internet, web")
pm_kw_list.append('2019-nCoV, telemedicine, cybercare')

question_list.append("How is artificial intelligence being used in real time health delivery")
kw_list.append("2019-nCoV, ai, real-time")
pm_kw_list.append('')

question_list.append("What adjunctive or supportive methods can help patients")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, adjunctive, supportive")
pm_kw_list.append('')

topic_area['What has been published about medical care?'] = list(zip(question_list,kw_list, pm_kw_list))

#6
#What do we know about diagnostics and surveillance?
question_list = []
kw_list = []
pm_kw_list = []
question_list.append("What diagnostic tests (tools) exist or are being developed to detect 2019-nCoV")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, diagnosis, tools, detetion")
pm_kw_list.append('2019-nCoV, diagnostic, tools, detetion')

question_list.append("What is being done to increase testing capacity or throughput")
kw_list.append("2019-nCoV, sars-cov-2, covid-19, diagnostic, testing, throughput")
pm_kw_list.append("2019-nCoV, testing capacity OR throughput")

question_list.append("What point of care tests are exist or are being developed")
kw_list.append("2019-nCoV, sars-cov-2, covid-19")
pm_kw_list.append("2019-nCoV, point-of-care")

question_list.append("What is the minimum viral load for detection")
kw_list.append("2019-nCoV, sars-cov-2, covid-19")
pm_kw_list.append("2019-nCoV, viral load")

question_list.append("What markers are used to detect or track COVID-19")
kw_list.append("2019-nCoV, sars-cov-2, covid-19")
pm_kw_list.append("2019-nCoV, markers")

topic_area['What do we know about diagnostics and surveillance?'] = list(zip(question_list,kw_list, pm_kw_list))



#7
#What has been published about information sharing and inter-sectoral collaboration?
question_list = []
kw_list = []
pm_kw_list = []
question_list.append('What collaborations are happening within the research community')
kw_list.append('inter-sectorial, international, collaboration, global, coronavirus, novel coronavirus, sharing')
pm_kw_list.append('2019-nCoV, collaboration, sharing')

topic_area['What has been published about information sharing and inter-sectoral collaboration?'] = list(zip(question_list,kw_list, pm_kw_list))


#8
#What has been published about ethical and social science considerations?
question_list = []
kw_list = []
pm_kw_list = []

question_list.append("What are the major ethical issues related pandemic outbreaks")
kw_list.append("ehtics, pandemic")
pm_kw_list.append("ethics, pandemic")

question_list.append("How do pandemics affect the physical and/or psychological health of doctors and nurses")
kw_list.append("2019-nCoV, sars-cov-2, covid-19, caregivers, health care workers")
pm_kw_list.append("2019-nCoV, physical OR psychological")

question_list.append("What strategies can help doctors and nurses cope with stress in a pandemic")
kw_list.append("2019-nCoV, sars-cov-2, covid-19, caregivers, health care workers")
pm_kw_list.append("2019-nCoV, physical OR psychological")

question_list.append("What factors contribute to rumors and misinformation")
kw_list.append("2019-nCoV, sars-cov-2, covid-19, social media")
pm_kw_list.append("2019-nCoV, misinformation OR social media")

topic_area['What has been published about ethical and social science considerations?'] = list(zip(question_list,kw_list, pm_kw_list))



#-1
# Other interesting Questions
question_list = []
kw_list = []
pm_kw_list = []
question_list.append("What is the immune system response to 2019-nCoV")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, immune, immune system, response, immunity, antibodies")
pm_kw_list.append('2019-nCoV, immune system, immunity, antibodie')

question_list.append("Can personal protective equipment prevent the transmission of 2019-nCoV")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, ppe, masks, gloves, face shields, gown, eye protection")
pm_kw_list.append('2019-nCoV, ppe')

question_list.append("Can 2019-nCoV infect patients a second time")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, reinfected, multiple infections, second time, permenant immunity")
pm_kw_list.append('2019-nCoV, reinfected')

topic_area['Other interesting Questions'] = list(zip(question_list,kw_list, pm_kw_list))

In [ ]:
for topic, question_list in topic_area.items():
  print(topic)
  answers = qa.ask(topic)
  qa.display_answers(answers[:5])  
  for q, kw, pm_kw in question_list:
    print(pm_kw + " " + q)
    try:
      answers = qa.ask(pm_kw + " " + q)
      qa.display_answers(answers[:5])
    except:
      print("No answer for context query")
      answers = qa.ask(q)
      qa.display_answers(answers[:5])

What is known about transmission, incubation, and environmental stability?


,Candidate Answer,Context,Confidence,Document Reference
0,has propensities and limitations,it has immediate implications for treating the sick and focuses the search for the source and route of transmission as each biological agent has propensities and limitations in the environments in which it can reside and the mechanisms by which it can be transmitted.,0.576933,22237
1,climate and weather factors are known to affect the transmission and distribution of infectious diseases,climate and weather factors are known to affect the transmission and distribution of infectious diseases and statistical and mathematical modelling are continuously developing to investigate the impact of weather and climate on water associated diseases.,0.295599,8919
2,the fingers are easily contaminated,the fingers are easily contaminated .,0.104039,28014
3,the only demonstrated mechanism of infectious transmission is direct contact between people. perhaps they have a limited epidemic potential. the infectious capacity of this virus is not accurately known,the only demonstrated mechanism of infectious transmission is direct contact between people. perhaps they have a limited epidemic potential. the infectious capacity of this virus is not accurately known .,0.005595,28014
4,a great capacity for environmental and person to person dissemination,"coronavirus occasionally produces diarrhea, but it gained importance as a result of severe acute respiratory syndrome (sars) outbreaks, originated by coronavirus with a great capacity for environmental and person to person dissemination .",0.003990,28014


2019-nCoV, transmission, shedding Is the virus transmitted by aerisol, droplets, food, close contact, fecal matter, or water


,Candidate Answer,Context,Confidence,Document Reference
0,"2019 n co v is a newly identified coronavirus with high similarity to sars co v. we performed a structural analysis of the receptor binding domain (rbd) of spike glycoprotein responsible for entry of coronaviruses into host cells. the rbds from the two viruses share 72 % identity in amino acid sequences, and molecular simulation reveals highly similar ternary structures. however, 2019 n co v has a distinct loop with flexible glycyl residues replacing rigid prolyl residues in sars co v. molecular modeling revealed that 2019 n co v rbd has a stronger interaction with angiotensin converting enzyme 2 (ace2). a unique phenylalanine f486 in the flexible loop likely plays a major role because its penetration into a deep hydrophobic pocket in ace2. ace2 is widely expressed with conserved primary structures throughout the animal kingdom from fish, amphibians, reptiles, birds, to mammals. structural analysis suggests that ace2 from these animals can potentially bind rbd of 2019 n co v, making them all possible natural hosts for the virus. 2019 n co v is thought to be transmitted through respiratory droplets","2019 n co v is a newly identified coronavirus with high similarity to sars co v. we performed a structural analysis of the receptor binding domain (rbd) of spike glycoprotein responsible for entry of coronaviruses into host cells. the rbds from the two viruses share 72 % identity in amino acid sequences, and molecular simulation reveals highly similar ternary structures. however, 2019 n co v has a distinct loop with flexible glycyl residues replacing rigid prolyl residues in sars co v. molecular modeling revealed that 2019 n co v rbd has a stronger interaction with angiotensin converting enzyme 2 (ace2). a unique phenylalanine f486 in the flexible loop likely plays a major role because its penetration into a deep hydrophobic pocket in ace2. ace2 is widely expressed with conserved primary structures throughout the animal kingdom from fish, amphibians, reptiles, birds, to mammals. structural analysis suggests that ace2 from these animals can potentially bind rbd of 2019 n co v, making them all possible natural hosts for the virus. 2019 n co v is thought to be transmitted through respiratory droplets .",0.5,4587
1,"2019 n co v is a newly identified coronavirus with high similarity to sars co v. we performed a structural analysis of the receptor binding domain (rbd) of spike glycoprotein responsible for entry of coronaviruses into host cells. the rbds from the two viruses share 72 % identity in amino acid sequences, and molecular simulation reveals highly similar ternary structures. however, 2019 n co v has a distinct loop with flexible glycyl residues replacing rigid prolyl residues in sars co v. molecular modeling revealed that 2019 n co v rbd has a stronger interaction with angiotensin converting enzyme 2 (ace2). a unique phenylalanine f486 in the flexible loop likely plays a major role because its penetration into a deep hydrophobic pocket in ace2. ace2 is widely expressed with conserved primary structures throughout the animal kingdom from fish, amphibians, reptiles, birds, to mammals. structural analysis suggests that ace2 from these animals can potentially bind rbd of 2019 n co v, making them all possible natural hosts for the virus. 2019 n co v is thought to be transmitted through respiratory droplets","2019 n co v is a newly identified coronavirus with high similarity to sars co v. we performed a structural analysis of the receptor binding domain (rbd) of spike glycoprotein responsible for entry of coronaviruses into host cells. the rbds from the two viruses share 72 % identity in amino acid sequences, and molecular simulation reveals highly similar ternary structures. however, 2019 n co v has a distinct loop with flexible glycyl residues replacing rigid prolyl residues in sars co v. molecular modeling revealed that 2019 n co v rbd has a stronger interaction with angio

2019-nCoV, incubation period How long is the incubation period for the virus


...

2019-nCoV, ppe Can personal protective equipment prevent the transmission of 2019-nCoV


,Candidate Answer,Context,Confidence,Document Reference
0,"prevention of 2019 n co v transmission, like any other infectious agent, involves minimizing risk of exposure","prevention of 2019 n co v transmission, like any other infectious agent, involves minimizing risk of exposure .",0.669898,10694
1,"the mode of transmission has implications for the types of personal protective equipment (ppe) needed to protect healthcare providers in the prehospital, clinic, and hospital settings","the mode of transmission has implications for the types of personal protective equipment (ppe) needed to protect healthcare providers in the prehospital, clinic, and hospital settings .",0.172026,10694
2,"on personal protection equipment and dress code modalities, implemented in our hospital to prevent further dissemination of covid 19 infection","the purpose of this article is to discuss the measures, with specific focus on personal protection equipment and dress code modalities, implemented in our hospital to prevent further dissemination of covid 19 infection .",0.090197,21943
3,that wearing personal protective equipment can effectively reduce or even block the spread of respiratory infectious diseases,"a systematic review based on randomized controlled trials confirmed that wearing personal protective equipment can effectively reduce or even block the spread of respiratory infectious diseases [ 19 ], thereby protecting medical personnel and other patients.",0.035261,13372
4,"the epicentre of the epidemic in china provided an early warning for europe to allow the preparation of preventive measures, and specific organisational processes were rapidly adopted. patients returning from china, and in particular from hubei province are quarantined for 14 days. anyone who develops symptoms that could suggest a coronavirus infection are encouraged to call a single emergency number and if covid 19 is suspected, they are managed at their location by a specialised medical team equipped with ppe to prevent viral contamination","the epicentre of the epidemic in china provided an early warning for europe to allow the preparation of preventive measures, and specific organisational processes were rapidly adopted. patients returning from china, and in particular from hubei province are quarantined for 14 days. anyone who develops symptoms that could suggest a coronavirus infection are encouraged to call a single emergency number and if covid 19 is suspected, they are managed at their location by a specialised medical team equipped with ppe to prevent viral contamination , and when necessary, hospitalised in an intensive care unit.",0.020850,24707


2019-nCoV, reinfected Can 2019-nCoV infect patients a second time


No answer for context query


,Candidate Answer,Context,Confidence,Document Reference
0,"for the third time in as many decades, a zoonotic coronavirus has crossed species to infect human populations","for the third time in as many decades, a zoonotic coronavirus has crossed species to infect human populations .",1.0,22590
